In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Data Source
#https://www.kaggle.com/nehalbirla/vehicle-dataset-from-cardekho/?select=Car+details+v3.csv
'''
## Task Details
The Objective - deeply analyze the dataset, by experimenting with a variety of ML models and technics
--- Main Tasks ---

1. Cleaning and organizing the data
2. Get an initial understanding of relations in the data 
3. Clustering – for categorizing data, set “strong” features
4. Regression – seeking the best model to determine “selling price”  
''' 

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import MeanShift
from sklearn.cluster import Birch
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import MiniBatchKMeans

import warnings
#warnings.filterwarnings("ignore")

In [ ]:
import os
os.getcwd()

In [ ]:
df = pd.read_csv('/kaggle/input/vehicle-dataset-from-cardekho/Car details v3.csv').drop('torque',axis = 1 ).dropna(how = 'all')

df.drop_duplicates(subset = ['name','year','selling_price', 'km_driven', 'seller_type','mileage'], keep = 'first')
df.sample(4)

In [ ]:
df.owner.unique()
# sum(df.owner.str.contains('First | Second | third', case = False))
df['owner'].replace({'First Owner': 1,'Second Owner':2, 'Third Owner':3,'Fourth & Above Owner':4, 'Test Drive Car':6}, inplace = True)

In [ ]:
df.owner.unique()

In [ ]:
# set the car age feature 

import datetime as dt
dt.datetime.today().year
df['car_age'] = dt.datetime.today().year - df.year 
df['car_age']
df.drop('year', axis = 1 ) 

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# set features as numeric
df['max_power'] = df['max_power'].str.strip('bhp')
df['max_power'] = df['max_power'].str.replace(" ", "")
df['max_power'].sort_values(ascending = False).head()#.astype('float64')
df['max_power'] = pd.to_numeric(df['max_power'],errors='coerce')

df['engine'] = df['engine'].str.replace("CC", "")
df['engine'] = pd.to_numeric(df['engine'], errors= 'coerce')
df['mileage'] = df['mileage'].str.replace("kmpl", "")
df['mileage'] = pd.to_numeric(df['mileage'], errors= 'coerce')
df['mileage'].sample(5)


In [ ]:
# Change name to brand
df['brand'] = df.name.str.split(' ').str[0]
df.drop(['year','name'], axis =1, inplace = True)

In [ ]:
# drop nulls
df.dropna(inplace = True)
# df.isnull().any()
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
# Shuffling the data
df.sample(frac = 1).reset_index(drop = True)

In [ ]:
df.describe()

### Some Descriptive Statistics

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(figsize = (10,5))
sns.heatmap(df.corr(), annot = True )

In [ ]:
plt.boxplot(df[['max_power','car_age', 'engine']])
plt.show()
df[['max_power','car_age', 'engine']].describe()

In [ ]:
fig, ax  = plt.subplots(figsize = (10,5)) 
plt.scatter(df.selling_price, df.max_power)
plt.ylabel('Max Power')
plt.xlabel('Selling Price')
plt.show()

### Categorial data into no.

In [ ]:
print('seller_type: ',df.seller_type.unique())
print('transmission: ',df.transmission.unique())
print('fuel: ',df.fuel.unique())

In [ ]:
from sklearn import preprocessing as pre
df.transmission = pre.LabelEncoder().fit_transform(df.transmission)
df.fuel = pre.LabelEncoder().fit_transform(df.fuel)


In [ ]:
print('transmission: ',df.transmission.unique())
print('fuel: ',df.fuel.unique())

In [ ]:
# One Hot Encoding - (for better nameing...)

df_temp = pd.get_dummies(df.seller_type)

print(df_temp.iloc[:,0:2].shape  , df.shape)
df  = pd.concat([df,df_temp.iloc[:,0:2]], axis = 1)
print(df.shape)

del df_temp
df.drop(['seller_type'],axis = 1, inplace= True)

In [ ]:
df  = pd.get_dummies(df, columns= ['brand'])
df.shape

### K-mean - Basic

In [ ]:
from sklearn.metrics import silhouette_score
model = KMeans(n_clusters=3,max_iter=1000, algorithm = 'full', tol=0.0001 ).fit(df)
model.labels_

In [ ]:
# The silhouette_score gives the average value for all the samples.
# This gives a perspective into the density and separation of the formed
# clusters
silhouette_avg = silhouette_score(df,model.labels_)
print("For n_clusters =", model.n_clusters,
      "The average silhouette_score is :", silhouette_avg)

### Hyper Tuning - the clustering models
#### standardize the data

In [ ]:
df_num_only = df.iloc[:,np.r_[0:2,5:9]]
df_full_feature = df
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df_std = sc.fit_transform(df_full_feature)
df_std_num = sc.fit_transform(df_num_only)
print('std values , all features ')
print(df_std)
print('\n std values , only numeric features ')
print(df_std_num)

#### K-mean

In [ ]:
from sklearn.model_selection import ParameterGrid

In [ ]:
best_score = -1
n_Kmeans =list(range(3,11,1))
n_Kmeans

In [ ]:
best_score = -1
best_no_Kmeans = 0
inter_no = 0
for i in n_Kmeans:
    for j in [200, 1000, 100000]:
        model = KMeans(n_clusters=i,max_iter= j , algorithm = 'full', tol=0.00001 ).fit(df_std_num)
        silhouette_avg = silhouette_score(df_std_num,model.labels_)
        print("For n_clusters =", model.n_clusters,'inter_no: ',model.max_iter,"The average silhouette_score is :", silhouette_avg)
        if silhouette_avg > best_score:
            best_score =  silhouette_avg
            best_no_Kmeans = model.n_clusters
            inter_no = model.max_iter

In [ ]:
print("Best n_clusters =", best_no_Kmeans,'Max_iter: ',model.max_iter," | Best silhouette_score is :", best_score)

In [ ]:
model = KMeans(n_clusters =  best_no_Kmeans ,max_iter=1000).fit(df)
KMeans_results = model.labels_

In [ ]:
len(KMeans_results)

#### MeanShift clustering

In [ ]:
from sklearn.cluster import estimate_bandwidth
estimate_bandwidth(df)

In [ ]:
model = MeanShift(bandwidth=estimate_bandwidth(df)).fit(df)

In [ ]:
model.labels_

In [ ]:
silhouette_score = silhouette_score(df, model.labels_)
silhouette_score

In [ ]:
print('silhouette_score: ' , silhouette_score, ' | No. of clusters: ', len(set(model.labels_)))

In [ ]:
len(set(model.labels_))

In [ ]:
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth
model = MeanShift(bandwidth=estimate_bandwidth(df_std_num)).fit(df_std_num)

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score = silhouette_score(df_std_num, model.labels_)
silhouette_score

In [ ]:
MeanShift_results = model.labels_

In [ ]:
print('silhouette_score: ' , silhouette_score, ' | No. of clusters: ', len(set(model.labels_)))

In [ ]:
set(model.labels_)

#### Birch clustering

In [ ]:
n_clusters =np.arange(3,10,1)
n_clusters

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import Birch
best_score = -1
best_no_clusters = 0
for i in n_clusters:
    model = Birch(n_clusters= i, threshold= 0.3).fit(df_std_num)
    print(i, model.labels_)
    silhouette_avg = silhouette_score(df_std_num,model.labels_)
    print("For n_clusters =", model.n_clusters,"The average silhouette_score is :", silhouette_avg)
    if silhouette_avg > best_score:
        best_score =  silhouette_avg
        best_no_clusters = model.n_clusters

In [ ]:
set(model.labels_)

In [ ]:
silhouette_score(df_std,model.labels_)

In [ ]:
print('Best No. of clusters: ', best_no_clusters," | Best silhouette_score is :", best_score)

In [ ]:
model = Birch(n_clusters= best_no_clusters, threshold= 0.3).fit(df_std_num)
Birch_results = model.labels_
len(Birch_results)

In [ ]:
set(model.labels_)

In [ ]:
#export base table
df.to_csv('Car_processed.csv', index= False)

In [ ]:
df_cl = df.copy()

In [ ]:
# carry 'Birch' features to nest step (regression)

df_cl['Birch'] = Birch_results
#df_cl['MeanShift'] = MeanShift_results
# df_cl['DBSCAN'] = DBSCAN_results
#df_cl['KMeans'] = KMeans_results
df_cl

In [ ]:
df_cl.pivot_table(columns= 'owner', index= 'Birch', values = 'selling_price', aggfunc='mean')

In [ ]:
#export table with Clusters assigned

df_cl.to_csv('Car_processed_cl.csv', index= False)

In [ ]:
ls

In [ ]:
del df

In [ ]:
df = pd.read_csv('Car_processed_cl.csv')

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
header_names = df.columns

In [ ]:
df1 = df[['selling_price', 'km_driven','mileage', 'engine', 'max_power', 'seats', 'car_age','owner']]
df1.head()
df2= df.drop(['selling_price', 'km_driven','mileage', 'engine', 'max_power', 'seats', 'car_age','owner'], axis=1)
df2.head()

In [ ]:
names = df1.columns
df1.sample(3)

In [ ]:
from sklearn.preprocessing import StandardScaler as sc
# sc = StandardScaler()
df1 = sc().fit_transform(df1)

df1 = pd.DataFrame(df1)

In [ ]:
df1.columns = names
df1

In [ ]:
df = pd.concat((df1,df2), axis = 1)
df.head()

In [ ]:
# The Birch classifier 

temp = pd.concat( [df2[['Birch']],df1],axis =1)

birch0 = temp.loc[temp.Birch ==0,:].drop(['Birch','km_driven'], axis =1)
birch1 = temp.loc[temp.Birch ==1,:].drop(['Birch','km_driven'], axis =1)
birch2 = temp.loc[temp.Birch ==2,:].drop(['Birch','km_driven'], axis =1)

fig = plt.figure(figsize =(12, 8))
 
# Creating axes instance
# ax = fig.add_axes([0, 0, 1, 1])
ax = fig.add_subplot(111)

# Creating plot
bp1 = ax.boxplot(birch0)
bp2 = ax.boxplot(birch1) 
bp3 = ax.boxplot(birch2,patch_artist=True)

# bp1 = box_plot(example_data1, 'red', 'tan')

for median in bp1['medians']:
    median.set(color ='Blue',
               linewidth = 4)
for median in bp2['medians']:
    median.set(color ='Orange',
               linewidth = 4)
for median in bp3['medians']:
    median.set(color ='Red',
           linewidth = 4)
plt.title('Birch clusters - Blue:0 , Orange:1, Red:2')

# # fill with colors
# colors = ['pink', 'lightblue', 'lightgreen']
# for bplot in (bp1, bp2,bp3):
#     for patch, color in zip(bplot['boxes'], colors):
#         patch.set_facecolor(color)


# show plot
plt.show(bp1)
plt.show(bp2)
plt.show(bp3)


In [ ]:
fig, ax  = plt.subplots(figsize = (10,5)) # set the frame
plt.scatter(df.car_age,df.selling_price)
plt.title('Selling Price Vs. Age')
plt.ylabel('selling_price')

In [ ]:
fig, ax  = plt.subplots(figsize = (10,5)) # set the frame
plt.scatter(df.seats,df.max_power)

plt.ylabel('selling_price')

In [ ]:
#! pip install fitter
from fitter import Fitter, get_common_distributions, get_distributions
sns.set_context('paper', font_scale= 1.5)
sns.displot(data=df, x = 'selling_price', kind='hist', bins = 100, aspect= 2)
plt.title('Selling Price Distribution')
plt.show()

In [ ]:
sns.set_context('paper', font_scale= 1.5)
sns.displot(data=df, x = 'car_age', kind='hist', bins = 50, aspect= 2)
plt.title('car_age Distribution')
plt.show()

In [ ]:
sns.set_context('paper', font_scale= 1.5)
sns.displot(data=df, x = 'km_driven', kind='hist', bins = 100, aspect= 2)
plt.title('km_driven Distribution')
plt.show()

In [ ]:
sns.set_context('paper', font_scale= 1.5)
sns.displot(data=df, x = 'mileage', kind='hist', bins = 100, aspect= 2)
plt.title('mileage Distribution')
plt.show()

In [ ]:
sns.set_context('paper', font_scale= 1.5)
sns.displot(data=df, x = 'max_power', kind='hist', bins = 100, aspect= 2)
plt.title('engine Distribution')
plt.show()

In [ ]:
sns.set_context('paper', font_scale= 1.5)
sns.displot(data=df, x = 'seats', kind='hist', bins = 100, aspect= 2)
plt.title('seats Distribution')
plt.show()

In [ ]:
sns.set_context('paper', font_scale= 1.5)
sns.displot(data=df, x = 'car_age', kind='hist', bins = 100, aspect= 2)
plt.title('car_age Distribution')
plt.show()

In [ ]:
for i in range(0,3):
    df  = df.sample(frac= 1).reset_index(drop = True) 
    print('reshuffle no.: ',i)

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(figsize = (8,4))
sns.heatmap(df1.corr(),vmin= -0.3, vmax=- 0.6, annot = True)

### Linear regression with one feature - selling price ~ max_power

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(figsize = (8,4))
sns.heatmap(df1.corr(), annot = True)

### Linear regression with one feature - selling price ~ max_power

In [ ]:
from sklearn.model_selection import train_test_split

x = df[['max_power']]
y = df[['selling_price']]

x_train,x_test,y_train, y_test = train_test_split(x,y, test_size= 0.3) 

In [ ]:
x_train.sample(5)
print(x_train.size, x_test.size, x_train.size+ x_test.size)

In [ ]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression().fit(x_train, y_train)

In [ ]:
linear_model.score(x_train, y_train)

In [ ]:
y_pred = linear_model.predict(x_test)
y_pred

In [ ]:
from sklearn.metrics import r2_score , mean_squared_error
print('Linear reg - selling price ~ max_power (one feature)')
print('r2_score: ', r2_score(y_test,y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))

### Linear regression with one feature - selling price ~engine

In [ ]:
plt.scatter(x_test,y_test)
plt.plot(x_test, y_pred, color = 'red')
plt.title('Linear regression with one feature - selling price ~ max_power')

plt.show()

### Linear regression with one feature - selling price ~engine

In [ ]:
x = df[['engine']]
y = df[['selling_price']]

x_train,x_test,y_train, y_test = train_test_split(x,y, test_size= 0.3) 

In [ ]:
linear_model = LinearRegression(normalize=True).fit(x_train,y_train)

In [ ]:
y_pred = linear_model.predict(x_test)

In [ ]:
print('Linear reg - selling price ~ engine (one feature)')
print('r2_score: ', r2_score(y_test,y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))

In [ ]:
fig , ax = plt.subplots()
plt.scatter(x_test,y_test)
plt.plot(x_test,y_pred, color= 'red')
plt.title('Linear regression with one feature - selling price ~ engine')
plt.show()

### Linear regression with all mumeric features

In [ ]:
df1.sample(7)

In [ ]:
X = df1.drop('selling_price', axis = 1)
Y = df1['selling_price']
X
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

linear_model2 = LinearRegression(normalize = True).fit(x_train,y_train)

y_pred = linear_model2.predict(x_test)

In [ ]:
coef  = pd.Series(linear_model2.coef_, x_train.columns)
print(coef)

In [ ]:
print('Linear reg - selling price ~ all Numeric Featurs')
print('r2_score: ', r2_score(y_test,y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(y_pred[0:300], label = 'pred')
plt.plot(list(y_test[0:300]), label = 'test')
plt.ylabel('selling_price')
plt.title('Linear regression - selling price ~ All Numeric (300)')
plt.legend()
plt.show()

In [ ]:
X = df.drop('selling_price', axis = 1)
Y = df['selling_price']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
linear_model3 = LinearRegression(normalize=True).fit(x_train,y_train)
y_pred = linear_model3.predict(x_test)

print('Linear reg - selling price ~ all Featurs')
print('r2_score: ', r2_score(y_test,y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))

In [ ]:
plt.figure(figsize = (10,5))
plt.plot(y_pred[0:300], label = 'pred')
plt.plot(list(y_test[0:300]), label = 'test')
plt.ylabel('selling_price')
#plt.legend()
plt.title('Linear regression - selling price ~ All Features (300)')
plt.show()

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

In [ ]:
b = df2.loc[:,('fuel','transmission','Dealer','Individual','Birch')]
a = df1.drop(['km_driven','seats'],axis =1)
df_n = pd.concat([a,b],axis =1)
del a,b
df_n.columns

In [ ]:
X = df_n.drop('selling_price', axis =1)
Y = df_n['selling_price']
x_train, x_test, y_train, y_test = train_test_split(X, Y , test_size= 0.3)

In [ ]:
lasso_model = Lasso(alpha= 0.9).fit(x_train, y_train)
y_pred = lasso_model.predict(x_test)

### **Hyper Parameters**
#### Linear LASSO

In [ ]:
param = {'alpha' : [float(i) for i in np.arange(0.01,1,0.1)]}
param

In [ ]:
grid_search = GridSearchCV(Lasso(), param, cv=10, return_train_score=True)
grid_search.fit(x_train, y_train)

grid_search.best_params_

In [ ]:
lasso_model = Lasso(alpha=grid_search.best_params_['alpha']).fit(x_train, y_train)

In [ ]:
y_pred = lasso_model.predict(x_test)
y_pred

In [ ]:
print('MSE_score: ', mean_squared_error(y_test, y_pred))

In [ ]:
result_dict = {}

result_dict['lasso_model_MSE'] = mean_squared_error(y_test, y_pred)
result_dict

plt.plot(y_test.values, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.show()

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet

grid_search = GridSearchCV(Ridge(), param, cv=10, return_train_score=True)
grid_search.fit(x_train, y_train)

grid_search.best_params_

In [ ]:
grid_search.best_params_['alpha']

#### Linear RIDGE

In [ ]:
Ridge()
ridge_model = Ridge(alpha=grid_search.best_params_['alpha'],normalize=True, max_iter=100000).fit(x_train, y_train)
y_pred = ridge_model.predict(x_test)
y_pred

In [ ]:
def get_results(model_name):
    print('MSE_score: ', mean_squared_error(y_test, y_pred))
    result_dict[model_name] = mean_squared_error(y_test, y_pred)
    plt.plot(y_test.values, label='Actual')
    plt.plot(y_pred, label='Predicted')
    plt.legend()
    plt.show()
    for key,value in result_dict.items():
        print(key,': ',value)

In [ ]:
get_results('Ridge_model_MSE')

#### Linear Elastic Net

In [ ]:
param = {}
param = {'alpha' : [float(i) for i in np.arange(0.5,1.7,0.3)]}
param['max_iter'] = [j for j in range(300,140400,40000)]
param['l1_ratio'] = [k for k in np.arange(0.2,1,0.2)]
param

In [ ]:
grid_search = GridSearchCV(ElasticNet(), param, cv=10, return_train_score=True)
grid_search.fit(x_train, y_train)

grid_search.best_params_

In [ ]:
grid_search.best_params_['l1_ratio']

In [ ]:
elastic_net_model = ElasticNet(alpha=grid_search.best_params_['alpha'] \
                               , max_iter=grid_search.best_params_['max_iter'],\
                               l1_ratio=grid_search.best_params_['l1_ratio']).fit(x_train, y_train)
y_pred = elastic_net_model.predict(x_test)
get_results('elastic_net_model_MSE')

#### KNN model

In [ ]:
from sklearn.neighbors import KNeighborsRegressor as knn

param = {}
param['n_neighbors'] = [i for i in range(2,22,2)]
param

In [ ]:
grid_search = GridSearchCV(knn(),param, cv =10, return_train_score= True)
grid_search.fit(x_train,y_train)
print(grid_search.best_score_)
grid_search.best_params_['n_neighbors']

In [ ]:
knn_model = knn(n_neighbors= grid_search.best_params_['n_neighbors']).fit(x_train,y_train)
y_pred = knn_model.predict(x_test)
get_results('knn_model_MSE')

#### Decision Tree model

In [ ]:
from sklearn.tree import DecisionTreeRegressor

param = {}
param['max_depth'] = [i for i in range(1,5)]
param

In [ ]:
grid_search = GridSearchCV(DecisionTreeRegressor(), param, cv=10)
grid_search.fit(x_train, y_train)
grid_search.best_params_ , grid_search.best_score_, grid_search.best_estimator_

In [ ]:
model_decision_tree = grid_search.best_estimator_.fit(x_train,y_train)
y_pred = model_decision_tree.predict(x_test)
get_results('decision_tree_model_MSE')

#### SVR model (SVM) - Support Vector Regression

In [ ]:
from sklearn.svm import SVR

param = {}
param['epsilon'] = [i for i in np.arange(0.02,0.7,0.1)]
param['C'] = [j for j in np.arange(0.1,1.2,0.3)]          # C is the penalty
param

In [ ]:
grid_search = GridSearchCV(SVR(), param, cv=10, return_train_score=True)
grid_search.fit(x_train, y_train)
grid_search.best_params_ , grid_search.best_estimator_

In [ ]:
model_SVR = SVR(kernel='linear', epsilon=grid_search.best_params_['epsilon'],\
                C=grid_search.best_params_['C']).fit(x_train,y_train)
y_pred = model_SVR.predict(x_test)
get_results('Support_Vector_Reg_model_MSE')

#### LARS model - Least Angel Regression
###### LARS Regression (Least Angle Regression Squers) <br>equel to step wise regression , fit small data with alot of featurs<br>puts the most importent coeficients featurs first

In [ ]:
from sklearn.linear_model import Lars

param = {}
param['n_nonzero_coefs'] = [i for i in range(1,11)]
param

In [ ]:
grid_search = GridSearchCV(Lars(), param, cv=10, return_train_score=True)
grid_search.fit(x_train, y_train)
grid_search.best_params_ , grid_search.best_estimator_

In [ ]:
model_LARS = grid_search.best_estimator_.fit(x_train,y_train)
y_pred = model_LARS.predict(x_test)
get_results('Least_Angel_Reg_model_MSE')

### RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rnd_forest_reg = RandomForestRegressor(n_estimators=600, max_leaf_nodes=12, n_jobs=-1).fit(x_train, y_train)

y_pred = rnd_forest_reg.predict(x_test)

get_results('Random_Forest_model_MSE')

In [ ]:
rnd_forest_reg.feature_importances_

In [ ]:
importent_features  = pd.DataFrame(rnd_forest_reg.feature_importances_, x_test.columns)
importent_features.columns =['importances']
importent_features = importent_features.sort_values('importances', ascending= False)

In [ ]:
importent_features['feature'] = importent_features.index
importent_features = importent_features.reset_index()
importent_features

In [ ]:
# fig, ax  = plt.subplots(figsize = (13,5)) # set the frame
# plt.bar(imported_features.feature, imported_features.importances ,color ='maroon')
# plt.title("importent_features")
# plt.show()

In [ ]:
rt = result_dict.copy()
# list(rt.items())
rt = pd.DataFrame(list(rt.items()),columns = ['Model','MSE_Result'])
rt['Model'] = rt['Model'].str.upper()
rt = rt.sort_values('MSE_Result')
rt

In [ ]:
fig, ax  = plt.subplots(figsize = (13,5)) # set the frame
plt.bar(rt.Model, rt.MSE_Result ,color = (0.5,0.1,0.5,0.6))
plt.title("Best MSE results")
fig.autofmt_xdate()
plt.show()

In [ ]:
from statsmodels.api import OLS
OLS(y_pred,x_test).fit().summary()

##   Gradient Boosting  
#### Random Forest  >  Least Angel Reg  >   Decision Tree

In [ ]:
nd_forest_reg = RandomForestRegressor(n_estimators=600, max_leaf_nodes=12, n_jobs=-1).fit(x_train, y_train)
y_pred = rnd_forest_reg.predict(x_train)

In [ ]:
y2 = y_train - y_pred
y2[:5]

In [ ]:
model_LARS = Lars(n_nonzero_coefs=9).fit(x_train,y_train)
y_pred = model_LARS.predict(x_train)

In [ ]:
y3 = y2 - y_pred
y3[:5]

In [ ]:
model_decision_tree = DecisionTreeRegressor(max_depth=4).fit(x_train,y3)

In [ ]:
y_pred = sum(model.predict(x_test) for model in (rnd_forest_reg, model_LARS, model_decision_tree))
get_results('Gardient_Boosting_3models_MSE')

In [ ]:
plt.figure(figsize = (15,5))
plt.plot(y_pred[0:200], label = 'pred')
plt.plot(list(y_test[0:200]), label = 'test')
plt.ylabel('selling_price_norm')
#plt.legend()
plt.title('Gradient Boosting (200) > Random Forest > Least Angel Reg > Decision Tree')
plt.show()

### Shap - features testing

In [ ]:
# import shap
# ex = shap.TreeExplainer(model_decision_tree)
# shap_values = ex.shap_values(x_test)
# shap.summary_plot(shap_values, x_test)
# # shap_values

*Gradient Boosting turn to be the best* : running :Random Forest  >  Least Angel Reg  >   Decision Tree